In [1]:
from database.models import  Scripts, MeroShareUser
from database.session import  get_db
from sqlalchemy.orm import selectinload
from sqlalchemy import select

In [2]:
async with get_db() as db:
    username = "3522757"
    meroshare_user = (await db.execute(select(MeroShareUser).filter(MeroShareUser.username == username))).scalars().first()
    meroshare_user = meroshare_user.to_dict()

In [ ]:
from nepse import Meroshare
meroshare = Meroshare(dp=meroshare_user['dp'], password=meroshare_user['password'], username=meroshare_user['username'])
await meroshare.start()

In [4]:
from utils import get_dir_path
from pathlib import Path
import pandas as pd
import os

current_date = pd.Timestamp.now().strftime('%Y-%m-%d')

dir_path = get_dir_path()
base_dir = Path(dir_path,'csv', meroshare_user['username'])
os.makedirs(base_dir, exist_ok=True)

pnl_dir = Path(base_dir,'PnL')
os.makedirs(pnl_dir, exist_ok=True)

wacc_rates_csv = Path(base_dir, 'Wacc Rates.csv')
pnl_xlsx = Path(base_dir,'P&L.xlsx')
pnl_pdf = Path(base_dir,'P&L.pdf')


wacc = pd.read_csv(Path(base_dir, 'My Wacc Report.csv'))
shares = pd.read_csv(Path(base_dir, 'My Shares Values.csv'))

In [5]:
import subprocess
if pnl_pdf.exists():
    subprocess.run(['mv',
                    f'{pnl_pdf}',
                    f'{pnl_dir}/{current_date}-P&L.pdf']
                )

In [6]:
merged_df = pd.merge(wacc, shares, left_on='Scrip Name', right_on='Scrip')
result = merged_df[['Scrip Name', 'Current Balance', 'WACC Rate']].copy()
result['WACC Rate'] = result['WACC Rate'].round(2)
result['Current Balance'] = result['Current Balance'].astype(float)
result = result.rename(columns={'Current Balance': 'Balance'})
result = result.rename(columns={'WACC Rate': 'WACC'})
result = result.rename(columns={'Scrip Name': 'Scrip'})

In [7]:
# from nepse import  refresh_all_script_details, refresh_script_detail
# for scrip in result['Scrip']:
#     print(f"Refreshing {scrip}...")
#     await refresh_script_detail(scrip)

Refreshing ADBL...
Refreshing AKPL...
Refreshing BFC...
Refreshing BNHC...
Refreshing CKHL...
Refreshing DOLTI...
Refreshing GBIME...
Refreshing GFCL...
Refreshing GLH...
Refreshing HRL...
Refreshing IGI...
Refreshing ILI...
Refreshing JBLB...
Refreshing KMCDB...
Refreshing KSBBL...
Refreshing MEL...
Refreshing MLBL...
Refreshing NABIL...
Refreshing NHPC...
Refreshing NIL...
Refreshing NLIC...
Refreshing NMB...
Refreshing PCBL...
Refreshing PHCL...
Refreshing RHGCL...
Refreshing SANVI...
Refreshing SCB...
Refreshing SGHC...
Refreshing SICL...
Refreshing SMFBS...
Refreshing SMJC...
Refreshing SONA...
Refreshing SWBBL...
Refreshing USHL...


In [8]:
async with get_db() as db:
    scripts = (await db.execute(select(Scripts).filter(Scripts.ticker.in_(result['Scrip'])).options(selectinload(Scripts.script_details)))).scalars().all()

In [9]:
high_low = result['Scrip'].apply(lambda name: next((script.script_details.week_52_high_low for script in scripts if script.ticker == name), None)) 

In [10]:
result['High'] = high_low.apply(lambda x: x.replace(',','').split(' - ')[0] if isinstance(x, str) else 0).astype(float)
result['LTP'] = result['Scrip'].apply(lambda name: next((script.script_details.last_traded_price for script in scripts if script.ticker == name), None)) 
result['Low'] = high_low.apply(lambda x: x.replace(',','').split(' - ')[1] if isinstance(x, str) else 0).astype(float)

result['Investment'] = result['Balance'] * result['WACC']
result['Investment'] = result['Investment'].round(2)

result['Current Value'] = result['Balance'] * result['LTP']
result['Current Value'] = result['Current Value'].round(2)

result['Profit/Loss'] = result['Current Value'] - result['Investment']
result['Profit/Loss'] = result['Profit/Loss'].round(2)

result['Diff %'] = (result['Profit/Loss'] / result['Investment']) * 100
result['Diff %'] = result['Diff %'].round(2)


result = result.sort_values(by=['Investment', 'Diff %'], ascending=[False, False])

totals = result[['Investment', 'Current Value', 'Profit/Loss']].sum()
totals['Scrip'] = 'Total'
totals['Balance'] = ''
totals['WACC'] = ''
totals['LTP'] = ''
totals['Diff %'] = (totals['Profit/Loss'] / totals['Investment']) * 100
totals['Diff %'] = round(totals['Diff %'],2)

result = pd.concat([result, totals.to_frame().T], ignore_index=True)

In [11]:
result.to_csv(wacc_rates_csv, index=False)
result.to_excel(pnl_xlsx, index=False)

In [12]:
import openpyxl
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
from openpyxl.worksheet.page import PageMargins

wb = openpyxl.load_workbook(pnl_xlsx)
ws = wb.active
ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
ws.page_margins = PageMargins(
    left=0.85,
    right=0.25,
    top=0.5,
    bottom=0.5,
)
ws.page_setup.horizontalCentered = True

ws.insert_rows(1)
ws.insert_rows(2)


title = f"[{meroshare_user['username']}] Profit and Loss Report ({current_date})"
ws.merge_cells('A1:J1')
title_cell = ws['A1']
title_cell.value = title
title_cell.alignment = Alignment(horizontal='center', vertical='center')
title_cell.font = Font(size=14, bold=True)

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for row in ws.iter_rows(min_row=3):
    for cell in row:
        cell.border = thin_border
# Set the width of columns
column_widths = {
    'A': 10,
    'B': 10,
    'C': 12,
    'D': 12,
    'E': 12,
    'F': 12,
    'G': 15,
    'H': 15,
    'I': 12,
    'J': 12
}

for col, width in column_widths.items():
    ws.column_dimensions[col].width = width

red_fill_1 = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")
red_fill_2 = PatternFill(start_color="FF6666", end_color="FF6666", fill_type="solid")
red_fill_3 = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

green_fill_1 = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
green_fill_2 = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")
green_fill_3 = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")

neutral_fill = PatternFill(start_color="FFFFCC", end_color="FFFFCC", fill_type="solid")

profit_loss_col = None
current_value_col = None
for idx, cell in enumerate(ws[3]):
    if cell.value == 'Diff %':
        profit_loss_col = idx + 1
    if cell.value == 'Balance':
        current_value_col = idx + 1

for row in ws.iter_rows(min_row=4, max_row=ws.max_row):
    profit_loss_value = row[profit_loss_col - 1].value
    current_value_value = row[current_value_col - 1].value
    if current_value_value and current_value_value == 10:
        fill = neutral_fill
        for cell in row:
            cell.fill = fill
            cell.border = thin_border
        continue
    if profit_loss_value:
        if profit_loss_value > 50:
            fill = green_fill_3
        elif profit_loss_value > 20:
            fill = green_fill_2
        elif profit_loss_value > 0:
            fill = green_fill_1
        elif profit_loss_value < -50:
            fill = red_fill_3
        elif profit_loss_value < -20:
            fill = red_fill_2
        elif profit_loss_value < 0:
            fill = red_fill_1
        else:
            fill = neutral_fill
        for cell in row:
            cell.fill = fill

wb.save(pnl_xlsx)

In [13]:
import subprocess
subprocess.run([
        'libreoffice',
        '--headless',
        '--convert-to', 'pdf',
        '--outdir', base_dir,
        pnl_xlsx
    ])

convert /home/kayc/Code/Python/price-tracker/csv/3522757/P&L.xlsx as a Calc document -> /home/kayc/Code/Python/price-tracker/csv/3522757/P&L.pdf using filter : calc_pdf_Export


CompletedProcess(args=['libreoffice', '--headless', '--convert-to', 'pdf', '--outdir', PosixPath('/home/kayc/Code/Python/price-tracker/csv/3522757'), PosixPath('/home/kayc/Code/Python/price-tracker/csv/3522757/P&L.xlsx')], returncode=0)